In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import yfinance as yf
import time
from functools import reduce
from typing import Union, List, Optional


In [2]:
df_final = pd.read_csv("C:/Users/CMert/Documents/Bachelor-projekt/Projektet/combined_stocks.csv")


In [3]:
df_final = df_final.drop(["Sector", "Industry", "Stocks", "Name", "Last Sale", "Net Change", "% Change", "Market Cap", "Country", "IPO Year"], axis=1)
df_final = df_final.dropna()

df_final = df_final.sort_values(by=["Symbol", "Date"])


In [4]:
# ensure Date is datetime
df_final['Date'] = pd.to_datetime(df_final['Date'])
df_final = df_final.sort_values(['Symbol', 'Date'])

# 2) Method A: groupby + pd.Grouper
df_monthly = (
    df_final
    .groupby([
        'Symbol',
        pd.Grouper(key='Date', freq='MS')   # MS = Month Start frequency
    ])
    .first()    # pick the first row in each (Symbol, Month) group
    .reset_index()
)


In [5]:
df_monthly

,Symbol,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,A,1999-11-01,32.546494,35.765381,28.612303,31.473534,26.511738,62546380.0,0.0,0.0
1,A,1999-12-01,30.177038,31.071173,29.953505,30.713518,25.871540,2957329.0,0.0,0.0
2,A,2000-01-01,56.330471,56.464592,48.193848,51.502148,43.382854,4674353.0,0.0,0.0
3,A,2000-02-01,47.389126,51.502148,47.389126,50.786839,42.780312,1404291.0,0.0,0.0
4,A,2000-03-01,74.749641,79.935623,74.570816,78.057938,65.752090,1037875.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1249064,ZYXI,2024-09-01,7.890000,8.090000,7.720000,7.890000,7.890000,225300.0,0.0,0.0
1249065,ZYXI,2024-10-01,8.080000,8.100000,7.710000,7.830000,7.830000,113800.0,0.0,0.0
1249066,ZYXI,2024-11-01,8.740000,8.780000,8.480000,8.760000,8.760000,132500.0,0.0,0.0
1249067,ZYXI,2024-12-01,8.320000,8.320000,7.950000,8.220000,8.220000,55800.0,0.0,0.0


In [43]:
list_of_stocks = df_monthly["Symbol"].unique()
symbols_df = pd.DataFrame(list_of_stocks, columns=["Symbol"])
symbols_df.to_csv("symbols.csv", index=False)

In [6]:
symbols_df = pd.read_csv("symbols.csv")
list_of_stocks = symbols_df["Symbol"].tolist()

In [13]:
import yfinance as yf
import pandas as pd
from typing import List, Optional

def fetch_and_flatten(
    symbols: List[str],
    period: str = "max",
    interval: str = "1d",
    auto_adjust: bool = True,
    actions: bool = True,
    **download_kwargs
) -> pd.DataFrame:
    """
    Download and flatten yfinance data for multiple symbols.
    
    Args:
        symbols: list of ticker strings, e.g. ['AAPL','MSFT'].
        period: same as yf.download (e.g. "1y","5d","max").
        interval: data granularity ("1d", "1wk", "1mo", etc.).
        auto_adjust: adjust OHLC for splits/divs.
        actions: include Dividends/Stock Splits columns.
        **download_kwargs: any extra args to pass to yf.download.
        
    Returns:
        A single DataFrame with:
          - Date as a DatetimeIndex
          - Columns: ['Open','High','Low','Close','Adj Close','Volume',...,'Symbol']
          - One row per date per symbol
    """
    dfs = []
    for sym in symbols:
        # pull the data
        df = yf.download(
            tickers=sym,
            period=period,
            interval=interval,
            auto_adjust=auto_adjust,
            actions=actions,
            **download_kwargs
        )
        if df.empty:
            print(f"Warning: no data for {sym}, skipping.")
            continue
        
        # flatten the MultiIndex if present
        if isinstance(df.columns, pd.MultiIndex):
            # level 0 is ['Open','High',...], level 1 is ticker
            # so drop ticker level
            df.columns = df.columns.droplevel(1)
        
        # add the symbol column
        df['Symbol'] = sym
        
        dfs.append(df)
    
    # combine all symbols into one DataFrame
    combined = pd.concat(dfs).sort_index()
    return combined

    


In [ ]:
# Dette kører yfinance for alle aktier

if __name__ == "__main__":
    symbols = list_of_stocks
    df_all1 = fetch_and_flatten(symbols, period="max", interval="1d")

In [21]:
df_all1 = df_all1.drop(["Capital Gains"], axis=1)
df_all1 = df_all1.dropna()

df_all1 = df_all1.sort_values(by=["Symbol", "Date"])

In [22]:
df_all1

Price,Close,Dividends,High,Low,Open,Stock Splits,Volume,Symbol
Date,,,,,,,,
1999-11-18,26.455530,0.0,30.063104,24.050482,27.357422,0.0,62546380,A
1999-11-19,24.275955,0.0,25.854267,23.937745,25.816687,0.0,15234146,A
1999-11-22,26.455530,0.0,26.455530,24.088060,24.839638,0.0,6577870,A
1999-11-23,24.050486,0.0,26.230062,24.050486,25.553641,0.0,5975611,A
1999-11-24,24.689325,0.0,25.215430,24.050483,24.125642,0.0,4843231,A
...,...,...,...,...,...,...,...,...
2025-04-11,2.190000,0.0,2.190000,2.030000,2.180000,0.0,144500,ZYXI
2025-04-14,2.210000,0.0,2.230000,2.120000,2.230000,0.0,195100,ZYXI
2025-04-15,2.200000,0.0,2.240000,2.150000,2.160000,0.0,143700,ZYXI


In [23]:
df_all1.to_csv("stock_data.csv")


In [2]:
df_final = pd.read_csv("C:/Users/CMert/Documents/Bachelor-projekt/Projektet/Til aktier/stock_data.csv")

In [3]:
df_final

,Date,Close,Dividends,High,Low,Open,Stock Splits,Volume,Symbol
0,1999-11-18,26.455530,0.0,30.063104,24.050482,27.357422,0.0,62546380,A
1,1999-11-19,24.275955,0.0,25.854267,23.937745,25.816687,0.0,15234146,A
2,1999-11-22,26.455530,0.0,26.455530,24.088060,24.839638,0.0,6577870,A
3,1999-11-23,24.050486,0.0,26.230062,24.050486,25.553641,0.0,5975611,A
4,1999-11-24,24.689325,0.0,25.215430,24.050483,24.125642,0.0,4843231,A
...,...,...,...,...,...,...,...,...,...
26126062,2025-04-11,2.190000,0.0,2.190000,2.030000,2.180000,0.0,144500,ZYXI
26126063,2025-04-14,2.210000,0.0,2.230000,2.120000,2.230000,0.0,195100,ZYXI
26126064,2025-04-15,2.200000,0.0,2.240000,2.150000,2.160000,0.0,143700,ZYXI
26126065,2025-04-16,2.150000,0.0,2.210000,2.130000,2.190000,0.0,110400,ZYXI


In [4]:
df_final['Date'] = pd.to_datetime(df_final['Date'])
df_final = df_final.sort_values(['Symbol', 'Date'])

# 2) Method A: groupby + pd.Grouper
df_monthly = (
    df_final
    .groupby([
        'Symbol',
        pd.Grouper(key='Date', freq='MS')   # MS = Month Start frequency
    ])
    .first()    # pick the first row in each (Symbol, Month) group
    .reset_index()
)

In [6]:
df_monthly

,Symbol,Date,Close,Dividends,High,Low,Open,Stock Splits,Volume
0,A,1999-11-01,26.455530,0.0,30.063104,24.050482,27.357422,0.0,62546380
1,A,1999-12-01,25.816685,0.0,26.117316,25.177844,25.365739,0.0,2957329
2,A,2000-01-01,43.290871,0.0,47.462125,40.510032,47.349387,0.0,4674353
3,A,2000-02-01,42.689613,0.0,43.290877,39.833617,39.833617,0.0,1404291
4,A,2000-03-01,65.612709,0.0,67.191024,62.681559,62.831874,0.0,1037875
...,...,...,...,...,...,...,...,...,...
1251080,ZYXI,2024-12-01,8.220000,0.0,8.320000,7.950000,8.320000,0.0,55800
1251081,ZYXI,2025-01-01,7.810000,0.0,8.120000,7.770000,8.120000,0.0,85600
1251082,ZYXI,2025-02-01,7.730000,0.0,7.760000,7.600000,7.720000,0.0,68200
1251083,ZYXI,2025-03-01,7.210000,0.0,7.340000,7.070000,7.260000,0.0,95800
